In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yt

import os
from typing import Callable

# suppress yt output
yt.set_log_level(50)

## HELPER FUNCTIONS ##
def get_box(rho):
    #box = np.stack(np.meshgrid(*[range(L) for L in rho.shape], indexing='ij'), axis=-1)-np.asarray(rho.shape)/2+0.5
    box = np.moveaxis(np.indices(rho.shape), 0, -1) - np.asarray(rho.shape)/2 + 0.5
    return box

def stokes_einstein(R, L, eta, kT, alpha=1):
    P = 1 - 2.84*R/L
    fn = (6+4*alpha)/(1+alpha)
    return kT*P/(fn*np.pi*eta*R)

def center_of_mass(rho):
    com = np.einsum('ijk,ijka', rho, get_box(rho))/np.sum(rho)
    return com

def gyration_tensor(rho, cm):
    box = np.array(rho.shape)
    r = get_box(rho) - cm
    r -= box*(r/box+np.sign(r)*0.5).astype(int) # minimum image convention
    S = np.einsum('ijk,ijka,ijkb', rho, r, r)/np.sum(rho)
    return S

def principal_radii(rho, cm, R):
    S = gyration_tensor(rho, cm)
    e, _ = np.linalg.eig(S)
    a, b, c = [ R*((e[i]*e[i])/(e[j]*e[k]))**(1/6) for i,j,k in [ np.roll(range(3),-n) for n in range(3) ] ]
    return a, b ,c

def droplet_radius_mass(density:np.ndarray)->float:
    center = tuple([l // 2 for l in density.shape])
    rho_d = density[center]
    rho_m = density[0, 0, 0]
    mass = np.sum(density - rho_m)
    R = (3./4./np.pi*mass/(rho_d - rho_m))**(1./3.)
    return R

def calculate_msd(ts, xs, tmax, scalar=True):
    ts = np.asarray(ts)
    xs = np.asarray(xs)
    dt = ts[1]-ts[0]
    ts = np.arange(tmax) 
    msd = np.array([ np.mean((xs[t:]-xs[:len(xs)-t])**2, axis=0) for t in ts ])
    if scalar: msd = msd.sum(axis=-1)
    return ts*dt, msd

def process_droplet(file:str, boxDims:np.ndarray, 
                    select_data:Callable[[dict], np.ndarray], 
                    img_filter = Callable[[np.ndarray], np.ndarray] | None)-> np.ndarray:
    ts = yt.load(file)
    processed_data = np.zeros((len(ts), 8))
    for j, ds in enumerate(ts):
        data = ds.covering_grid(level=0, left_edge=[0.0, 0.0, 0.0], dims=boxDims)
        data_select = select_data(data)
        filtered = data_select if filter is None else img_filter(data_select)
        processed_data[j, 0] = int(ds.current_time)
        processed_data[j, 1] = droplet_radius_mass(filtered)
        processed_data[j, 2:5] = center_of_mass(filtered)
        processed_data[j, 5:] = principal_radii(filtered, processed_data[j, 2:5], processed_data[j, 1])
    return processed_data

def plot_diffusion(ax, df, tau, label, marker, color, ls):
    L = df[['nx', 'ny', 'nz']].iloc[0].values.min()
    R = df.R.mean()
    eta = df.rho0.values[0]*(1/3)*(df.tau_r.values[0] - 1/2) 
    kT = df.kT.values[0]
    D = stokes_einstein(R, L, eta, kT)
    ts = df.t
    xs = df.cms_x - df[df.t==df.t.min()].cms_x.values
    ys = df.cms_y - df[df.t==df.t.min()].cms_y.values
    zs = df.cms_y - df[df.t==df.t.min()].cms_y.values
    rs = np.stack([xs, ys, zs]).T
    #print(rs.max())
    ts, msd = calculate_msd(ts, rs, tau)
    ax.plot(ts, msd, ls='-', c="green")
    Dfit = np.polyfit(ts, msd, deg=1)[0]/6
    ax.plot(ts[::10], msd[::10], marker, mfc='none', c=color, label=label)
    ax.plot(ts, 6*Dfit*ts, marker = "None", mfc='none', c=color, ls = ls)
    ax.plot(ts, 6*D*ts, ls='-', c=color)

    # print(f"Dse:{D:.4e}, Db:{Dfit:.4e}, diff:{(Dfit - D)*100/D:.2f}%")
    return D, Dfit
## HELPER FUNCTIONS ##

xdg_data_path = "/usa/xdengae/Binary-Fluctuating-Lattice-Boltzmann/data_droplet_density_1.00_alpha0_4.00_r0.200_size64-64-64/lbm_data_shshan_alpha0_4.00_xi_5.0e-05_size64-64-64_continue"

data_preprocess = lambda data: data['rho']
img_filter = lambda data: np.where(data > 0.06, data, 0)

xdg_boxDims = np.ones(3)*64
kT = 5e-5
rho0 = 1.0
tau_r = 1.0
tau_p = 1.0
gbr = 4.0
R_prop = 0.2
tau = 100

outfile = f"./MSD-rho_{rho0}-taur_{tau_r}-gbr_{gbr}-kbt_{kT}-R_{R_prop}.csv"
header = ['t', 'R', 'cms_x', 'cms_y', 'cms_z', 'a', 'b', 'c']

if os.path.exists(outfile):
    xdg_df = pd.read_csv(outfile, index_col = 0)
else:
    processed_data = process_droplet(f"{xdg_data_path}/plt*", xdg_boxDims, data_preprocess, img_filter)
    xdg_df = pd.DataFrame(processed_data, columns = header)
    xdg_df['kT'] = kT
    xdg_df['R0'] = xdg_df.R.values[0]
    xdg_df[['rho0', 'tau_r', 'tau_p']] = np.array([rho0, tau_r, tau_p])
    xdg_df[['nx', 'ny', 'nz']] = xdg_boxDims
    xdg_df.drop(0, inplace = True)
    xdg_df.to_csv(outfile)

colors = [ 'tab:blue', 'tab:green', 'tab:red' ]
marker = [ 'o', 's', 'd' ]
lss = ["-.", "--", ":"]

fig, axs = plt.subplots(1,2, figsize = (10,5))

ax = axs[0]
D, Dfit = plot_diffusion(axs[0], xdg_df, tau, label=f'$kT={kT:.0e}$', marker=marker[0], color=colors[0], ls = lss[0])
print(f"Dse:{D:.4e}, Db:{Dfit:.4e}, diff:{(Dfit - D)*100/D:.2f}%")

ax.set_xlabel(r'${\tau}/{\Delta t}$')
ax.set_ylabel(r'${\langle(\mathrm{r(t+\tau)}-\mathrm{r(t)})^2\rangle}/{(\Delta x)^2}$')
ax.set_title("MSD of droplet")

ax.text(
    0.73, 0.77,
    '○ Data\n—  Theoretical\n—. fit',
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment='top',
    bbox=dict(boxstyle='round,pad=0.4', facecolor='white', alpha=0.9, edgecolor='grey')
)

ax = axs[1]

ax.plot(xdg_df.t, xdg_df.R)

ax.set_xlabel(r'${t}/{\Delta t}$')
ax.set_ylabel(r"$R_d \Delta x$")
ax.set_title("Droplet radius fluctuations")

plt.close()
fig.tight_layout()
fig.savefig("./MCMP_SD_MSD_comparison.png", dpi = 300)

0.3504209764288769
Dse:9.2952e-07, Db:9.6660e-07, diff:3.99%
